In [ ]:
!pip install tensorflow==2.7.0
!pip install scikeras

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import pandas as pd
import numpy as np
import sklearn
import matplotlib as plt
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing import sequence 
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Embedding, LSTM
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold, ShuffleSplit, StratifiedKFold, StratifiedShuffleSplit, TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import math
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

Data Collection

In [ ]:
train = pd.read_csv('/kaggle/input/predicting-the-sales-of-products-of-a-retail-chain/train_data.csv')

print(train.head())

import datetime
lag_size = 28
print('Forecast lag size: ', lag_size)

Data Clearning

In [ ]:
train = train[(train['date'] >= '2013-01-01')]
train_gp = train.sort_values('date').groupby(['product_identifier', 'outlet', 'date'], as_index=False)
train_gp = train_gp.agg({'sales':['mean']})
train_gp.columns = ['product_identifier', 'outlet', 'date', 'sales']
train_gp.head()

scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(np.array(train_gp['sales']).reshape(-1,1))
normalized = scaler.transform(np.array(train_gp['sales']).reshape(-1,1))
print('Min: %f, Max: %f' % (scaler.data_min_, scaler.data_max_))
train_gp['sales'] = normalized.copy()

Transform the data into a time series problem

In [ ]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

Determine the window and lag size

In [ ]:
window = 30
lag = lag_size
series = series_to_supervised(train_gp.drop('date', axis=1), window=window, lag=lag)
series.head()

Transform the data into window t-Number

In [ ]:
last_product_identifier = 'product_identifier(t-%d)' % window
last_outlet = 'outlet(t-%d)' % window
series = series[(series['outlet(t)'] == series[last_outlet])]
series = series[(series['product_identifier(t)'] == series[last_product_identifier])]
series.head()

Drop columns not used

In [ ]:
columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['product_identifier', 'outlet']]
for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['product_identifier', 'outlet']]
series.drop(columns_to_drop, axis=1, inplace=True)
series.drop(['product_identifier(t)', 'outlet(t)'], axis=1, inplace=True)
series.head()

Split data in Train and Validation

In [ ]:
labels_col = 'sales(t+%d)' % lag_size
labels = series[labels_col]
series = series.drop(labels_col, axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(series, labels.values, test_size=0.2, random_state=4)
X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.25, random_state=4)

print('Train set shape', X_train.shape)
print('Validation set shape', X_valid.shape)
X_train.head()

Transform train and validation data into series

In [ ]:
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1], 1))
print('Train set shape', X_train_series.shape)
print('Validation set shape', X_valid_series.shape)

Model tunning

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.experimental import enable_halving_search_cv  
from sklearn.model_selection import HalvingGridSearchCV

def create_model():
 # create model
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(50, activation='relu', input_shape=(X_train_series.shape[1], X_train_series.shape[2]), dropout=0.2))
    model.add(tf.keras.layers.Dense(1))
 # Compile model
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return model

# define the grid search parameters
batch_size = [5, 10, 30, 60]
epochs = [20]
param_grid = dict(batch_size=batch_size, epochs=epochs)
model_new = KerasClassifier(model=create_model, verbose=2)
grid = HalvingGridSearchCV(estimator=model_new, param_grid=param_grid,cv=5, scoring='accuracy')

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_valid_encoded = label_encoder.fit_transform(Y_valid)

grid_result = grid.fit(X_train_series, Y_train_encoded, validation_data=(X_valid_series, Y_valid_encoded))
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Build the LSTM Model

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='relu', input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(1))
model_lstm.summary()
model_lstm.compile(optimizer='adam', loss='mse')

Train the lstm model with the best parameters

In [ ]:
lstm_history = model_lstm.fit(X_train_series, Y_train, validation_data=(X_valid_series, Y_valid), epochs=20, verbose=2, batch_size=30)

Performance measurement

In [ ]:
plt.plot(lstm_history.history['loss'])
plt.plot(lstm_history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train Loss', 'Validation Loss'], fancybox=True, shadow=True)
plt.show()

Performance in validation set

In [ ]:
# change back to min max
predicted_sales = model_lstm.predict(X_valid_series)

plt.plot(scaler.inverse_transform(np.array(Y_valid).reshape(-1,1)).flatten(), color='black', label=f"Actual Sales")
plt.plot(scaler.inverse_transform(np.array(predicted_sales).reshape(-1,1)).flatten(), color= 'green', label="Predicted Sales")
plt.title("Sales vs Predicted Sales")
plt.xlabel("Days in test period")
plt.ylabel("Volume")
plt.legend(fancybox=True, shadow=True)
plt.show()

RSME and MAPE

In [ ]:
def smape(a, f):
    return 1/len(a) * np.sum(2*np.abs(f-a)/(np.abs(a)+np.abs(f))*100)
print("{:.4}%".format(smape(scaler.inverse_transform(np.array(Y_valid).reshape(-1,1)).flatten(),scaler.inverse_transform(np.array(predicted_sales).reshape(-1,1)).flatten())))

In [ ]:
from sklearn.metrics import mean_squared_error

print(mean_squared_error(scaler.inverse_transform(np.array(Y_valid).reshape(-1,1)).flatten(),scaler.inverse_transform(np.array(predicted_sales).reshape(-1,1)).flatten()))

Epoch Batch

10 60 138.0% 29.166570980057436 with dropout

50 10 142.7% 29.5902925577321

50 60 139.8% 29.875080899034234

100 10 152.5% 8.927442666849982

10 30 159.0% 7.943576742473504

20 30 159.7% 7.812799257608862

50 30 167.6% 7.98768476096198

Performance in Test data

In [ ]:
X_test_series = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

print('Test shape', X_test_series.shape)
print('Test shape', Y_test.shape)

In [ ]:
predicted_sales_test = model_lstm.predict(X_test_series)

plt.plot(scaler.inverse_transform(np.array(Y_test).reshape(-1,1)).flatten(), color='black', label=f"Actual Sales")
plt.plot(scaler.inverse_transform(np.array(predicted_sales_test).reshape(-1,1)).flatten(), color= 'green', label="Predicted Sales")
plt.title("Test Sales vs Predicted Sales")
plt.xlabel("Days in test period")
plt.ylabel("Volume")
plt.legend(fancybox=True, shadow=True)
plt.show()

In [ ]:
print("{:.4}%".format(smape(scaler.inverse_transform(np.array(Y_test).reshape(-1,1)).flatten(),scaler.inverse_transform(np.array(predicted_sales_test).reshape(-1,1)).flatten())))

In [ ]:
print(mean_squared_error(scaler.inverse_transform(np.array(Y_test).reshape(-1,1)).flatten(),scaler.inverse_transform(np.array(predicted_sales_test).reshape(-1,1)).flatten()))

Test Performance

10 30 159.0% 8.672678367259874

10 60 134.1% 32.02048404856427  with dropout

20 30 159.7% 8.57719643359611

50 60 135.9% 32.79656928199366

50 10 138.7% 32.48762213712006
50 30 167.5% 8.7616621717911